## 21.Subshells

Running a shell script launches a new process, a subshell.

#### Definition: A subshell is a child process launched by a shell (or shell script).

A subshell is a separate instance of the command processor -- the shell that gives you the prompt at the console or in an xterm window. Just as your commands are interpreted at the command-line prompt, similarly does a script batch-process a list of commands. Each shell script running is, in effect, a subprocess (child process) of the parent shell.

A shell script can itself launch subprocesses. These subshells let the script do parallel processing, in effect
executing multiple subtasks simultaneously.

Now, run the script:  
sh subshell-test.sh  

And, while the script is running, from a different xterm:

In [9]:
ps -ef | grep -v grep | egrep "UID|subshell-test.sh" --color=auto

UID         PID   PPID  C STIME TTY          TIME CMD
liheyi    94934  94301  0 11:52 pts/2    00:00:00 bash subshell-test.sh
liheyi    94935  94934 76 11:52 pts/2    00:00:38 bash subshell-test.sh


Analysis:  
PID 94934, the script, launched PID 94935, the subshell.

In general, an external command in a script forks off a subprocess, whereas a Bash builtin does not.   
For this reason, builtins execute more quickly and use fewer system resources than their external command equivalents.

### Command List within Parentheses

##### ( command1; command2; command3; ... )
A command list embedded between parentheses runs as a subshell.

Variables in a subshell are not visible outside the block of code in the subshell.  
They are not accessible to the parent process, to the shell that launched the subshell.   
These are, in effect, variables local to the child process.

#### Example 21-1. Variable scope in a subshell

In [10]:
cat subshell.sh

#!/bin/bash
# subshell.sh

echo

echo "We are outside the subshell."
echo "Subshell level OUTSIDE subshell = $BASH_SUBSHELL"
# Bash, version 3, adds the new         $BASH_SUBSHELL variable.
echo

outer_variable=Outer
global_variable=
#  Define global variable for "storage" of
#+ value of subshell variable.
(
echo "We are inside the subshell."
echo "Subshell level INSIDE subshell = $BASH_SUBSHELL"
inner_variable=Inner

echo "From inside subshell, \"inner_variable\" = $inner_variable"
echo "From inside subshell, \"outer\" = $outer_variable"
global_variable="$inner_variable"  #  Will this allow "exporting"
                                   #+ a subshell variable?
)
echo

echo "We are outside the subshell."
echo "Subshell level OUTSIDE subshell = $BASH_SUBSHELL"
echo

if [ -z "$inner_variable" ]
then
    echo "inner_variable undefined in main body of shell"
else
    echo "inner_variable defined in main body of shell"
fi

echo "From main body of shell, \"inner_variable\" = $inner_variable"

In [11]:
./subshell.sh


We are outside the subshell.
Subshell level OUTSIDE subshell = 0

We are inside the subshell.
Subshell level INSIDE subshell = 1
From inside subshell, "inner_variable" = Inner
From inside subshell, "outer" = Outer

We are outside the subshell.
Subshell level OUTSIDE subshell = 0

inner_variable undefined in main body of shell
From main body of shell, "inner_variable" = 
global_variable = 

---------------------------------
$var INSIDE subshell = 42
$var OUTSIDE subshell = 41


#### Definition: 
The scope of a variable is the context in which it has meaning, in which it has a value that can be referenced. For example, the scope of a local variable lies only within the function, block of code, or subshell within which it is defined, while the scope of a global variable is the entire script in which it appears.

While the \$BASH_SUBSHELL internal variable indicates the nesting level of a subshell,  
the \$SHLVL variable shows no change within a subshell.

In [12]:
echo " \$BASH_SUBSHELL outside subshell = $BASH_SUBSHELL"

 $BASH_SUBSHELL outside subshell = 0


In [13]:
( echo " \$BASH_SUBSHELL inside subshell = $BASH_SUBSHELL" )

 $BASH_SUBSHELL inside subshell = 1


In [14]:
( ( echo " \$BASH_SUBSHELL inside nested subshell = $BASH_SUBSHELL" ) )

 $BASH_SUBSHELL inside nested subshell = 2


In [15]:
echo " \$SHLVL outside subshell = $SHLVL"

 $SHLVL outside subshell = 3


In [16]:
( echo " \$SHLVL inside subshell = $SHLVL" )

 $SHLVL inside subshell = 3


Directory changes made in a subshell do not carry over to the parent shell.

#### Example 21-2. List User Profiles

In [17]:
cat allprofs.sh

#!/bin/bash
# allprofs.sh: Print all user profiles.

# This script written by Heiner Steven, and modified by the document author.

FILE=.bashrc   #  File containing user profile,
               #+ was ".profile" in original script.
for home in `awk -F: '{print $6}' /etc/passwd`
do
  [ -d "$home" ] || continue   # If no home directory, go to next.
  [ -r "$home" ] || continue   # If not readable, go to next.
  (cd $home; [ -e $FILE ] && ls -l $FILE)
done

#  When script terminates, there is no need to 'cd' back to original directory,
#+ because 'cd $home' takes place in a subshell.
exit 0


In [18]:
sudo ./allprofs.sh

-rw-r--r-- 1 root root 3106  2月 20  2014 .bashrc
-rw-r--r-- 1 liheyi liheyi 3681  7月  7 15:27 .bashrc


A subshell may be used to set up a "dedicated environment" for a command group.

In [ ]:
COMMAND1
COMMAND2
COMMAND3
(
  IFS=:
  PATH=/bin
  unset TERMINFO
  set -C
  shift 5
  COMMAND4
  COMMAND5
  exit 3 # Only exits the subshell!
)
#  The parent shell has not been affected, 
#+ and the environment is preserved.
COMMAND6
COMMAND7

As seen here,  
the exit command only terminates the subshell in which it is running, not the parent shell or script.

One application of such a "dedicated environment" is testing whether a variable is defined.

In [19]:
if (set -u; : $variable) 2> /dev/null
then
  echo "Variable is set."
else
  echo "Variable is not set."
fi

Variable is not set.


Another application is checking for a lock file:

In [22]:
if (set -C; : > lock_file) 2> /dev/null
then
  echo "lock_file didn't exist: no user running the script"
else
  echo "Another user is already running that script."
exit 65
fi

lock_file didn't exist: no user running the script


Processes may execute in parallel within different subshells.   
This permits breaking a complex task into subcomponents processed concurrently.

#### Example 21-3. Running parallel processes in subshells

In [ ]:
(cat list1 list2 list3 | sort | uniq > list123) &
(cat list4 list5 list6 | sort | uniq > list456) &
# Merges and sorts both sets of lists simultaneously.
# Running in background ensures parallel execution.
#
# Same effect as
#   cat list1 list2 list3 | sort | uniq > list123 &
#   cat list4 list5 list6 | sort | uniq > list456 &

wait # Don't execute the next command until subshells finish.

diff list123 list456

Redirecting I/O to a subshell uses the "|" pipe operator, as in ls -al | (command).

In [23]:
ls -al | wc -l

21


###### { command1; command2; command3; . . . commandN; }
A code block between curly brackets does not launch a subshell.

In [24]:
var1=23
echo "var1 = $var1"

var1 = 23


In [25]:
{ var1=76;}
echo "var1 = $var1"

var1 = 76
